In [17]:
import cv2
import numpy as np
import mediapipe as mp
import screen_brightness_control as sbc
import subprocess
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL

In [18]:
def main():
    print("Starting the brightness control application...")
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(
        IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volRange = volume.GetVolumeRange()
    minVol , maxVol, _ = volRange
    print(f"Volume range: {volRange}")
    cap=cv2.VideoCapture(0)
    #### setting up model
    mp_hands=mp.solutions.hands
    hands=mp_hands.Hands(max_num_hands=2,static_image_mode=False,model_complexity=1,min_detection_confidence=0.75,min_tracking_confidence=0.75)
    draw=mp.solutions.drawing_utils
    try:
        while True:
            ret,frame=cap.read()
            if not ret:
                break


            frame=cv2.flip(frame,1)
            frameRGB=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            processed=hands.process(frameRGB)
            print(processed.multi_hand_landmarks)
            if processed.multi_hand_landmarks:
                left_hand_mark_list, right_hand_mark_list = get_left_right_landmarks(frame, processed,draw,mp_hands)

                if left_hand_mark_list:
                    left_distance= get_distance(frame,left_hand_mark_list)
                    bright_level=np.interp(left_distance,[0, 220], [0, 100])
                    sbc.set_brightness(int(bright_level))
                if right_hand_mark_list:
                    right_distance= get_distance(frame,right_hand_mark_list)
                    volume_level=np.interp(right_distance,[50, 220], [minVol, maxVol])
                    volume.SetMasterVolumeLevel(volume_level, None)
                    
            cv2.imshow("frame",frame)


            if cv2.waitKey(1) & 0xFF == 27:
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)

def get_left_right_landmarks(frame, processed,draw,hands):
    left_hand_mark_list = []
    right_hand_mark_list = []
    for hand_lm in processed.multi_hand_landmarks:
        for idx, lm in enumerate(hand_lm.landmark):
            height, width, _ = frame.shape
            x,y= int(lm.x * width), int(lm.y * height)

            if idx==4  or idx==8:
                landmark=[idx,x,y]

                if hand_lm==processed.multi_hand_landmarks[0]:
                    left_hand_mark_list.append(landmark)
                elif hand_lm==processed.multi_hand_landmarks[1]:
                    right_hand_mark_list.append(landmark)
        draw.draw_landmarks(frame, hand_lm, hands.HAND_CONNECTIONS)

    return left_hand_mark_list, right_hand_mark_list

def get_distance(frame,hand_mark_list):
    if len(hand_mark_list) == 2:
        x1, y1 = hand_mark_list[0][1], hand_mark_list[0][2]
        x2, y2 = hand_mark_list[1][1], hand_mark_list[1][2]
        distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        cv2.circle(frame, (x1, y1), 5, (0, 255, 0), cv2.FILLED)
        cv2.circle(frame, (x2, y2), 5, (0, 0, 255), cv2.FILLED)
        cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        mid_x = (x1 + x2) // 2
        mid_y = (y1 + y2) // 2
        cv2.putText(frame, f"{distance:.2f}", (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX,0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Distance: {distance:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        return distance
    return None


def set_external_brightness(level):
    """
    Set brightness on external monitor using ddcctl.
    Level: 0–100
    """
    level = max(0, min(int(level), 100))
    try:
        subprocess.run(["ddcctl", "-d", "1", "-b", str(level)], check=True)
        print(f"Set brightness to {level}%")
    except subprocess.CalledProcessError as e:
        print("Failed to set brightness:", e)


def get_mac_brightness():
    result = subprocess.run(["brightness", "-l"], capture_output=True, text=True)
    for line in result.stdout.splitlines():
        if "brightness" in line:
            print(line)
            print("Brightness level found:", line.split()[-1])
            return float(line.split()[-1])
    return None

In [22]:
if __name__ == "__main__":
    main();

Starting the brightness control application...
Volume range: (-65.25, 0.0, 0.03125)
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.0980384126
  y: 1.07054532
  z: 6.13146199e-007
}
landmark {
  x: 0.157314733
  y: 1.05343056
  z: -0.0418405458
}
landmark {
  x: 0.207165956
  y: 0.998263597
  z: -0.0708191469
}
landmark {
  x: 0.243039012
  y: 0.945367396
  z: -0.0953517556
}
landmark {
  x: 0.278407604
  y: 0.902374685
  z: -0.120174117
}
landmark {
  x: 0.143289328
  y: 0.910325
  z: -0.0613986589
}
landmark {
  x: 0.156726137
  y: 0.824383318
  z: -0.0956988782
}
landmark {
  x: 0.168093443
  y: 0.763727784
  z: -0.119229265
}
landmark {
  x: 0.175589532
  y: 0.709636688
  z: -0.133695289
}
landmark {
  x: 0.0951616094
  y: 0.886576951
  z: -0.0559860393
}
landmark {
  x: 0.0988164246
  y: 0.787247241
  z: -0.0833858922
}
landmark {
 